# Домашнее задание 2

В этом задании:

1. Сделаем регрессию над данными через scikit-learn: сначала через регресию, потом через бустинг.
2. Сравним результаты с константным предсказанием.
3. Сделаем нейронную сеть на полносвязных слоях, обучим над теми же данными - и сравним с лин. регрессией и бустингом.



In [13]:
import pandas as pd
import numpy as np
import random

# Для воспроизводимости
seed = 0
np.random.seed(seed)
random.seed(seed)

In [23]:
df = pd.read_csv("insurance.csv")

### Задание №1:
Cделайте train/test split на данных в пропорции 0.2/0.8, залейте в лмс код, который в `df_train`, `df_test`
сохранит датафрейм с тренировочными и тестовыми данными соответственно.

P.S Использовать train_test_split из scikit-learn запрещено - разбивайте вручную через индексы.

In [24]:
df.shape

(1338, 7)

In [25]:
idx_test = np.random.choice(df.shape[0], size=int(0.2 * df.shape[0]), replace=False)
df_test = df.iloc[idx_test]
df_train = df.drop(index=idx_test)

In [71]:
df_train = df.iloc[:int(len(df) * 0.8), :]
df_test = df.iloc[int(len(df) * 0.8):, :]

### Задание №2:
Сделайте `OHE` на колонки sex, region, smoker.

Нужно сделать как на train, так и на test датасете.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.
При этом в `df_train`, `df_test` должны остаться старые колонки - т.е. его нужно обогатить.


In [26]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [27]:


# Создаём OHE и обучаем на train-датасете
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  
ohe.fit(df_train[['sex', 'region', 'smoker']])  

# Трансформируем train и test
df_train_ohe = ohe.transform(df_train[['sex', 'region', 'smoker']])
df_test_ohe = ohe.transform(df_test[['sex', 'region', 'smoker']])

# Создаём DataFrame с OHE-признаками
ohe_columns = ohe.get_feature_names_out(['sex', 'region', 'smoker'])
df_train_ohe = pd.DataFrame(df_train_ohe, columns=ohe_columns, index=df_train.index)
df_test_ohe = pd.DataFrame(df_test_ohe, columns=ohe_columns, index=df_test.index)

# Обогащаем исходные DataFrame
df_train = pd.concat([df_train, df_train_ohe], axis=1)
df_test = pd.concat([df_test, df_test_ohe], axis=1)


### Задание №4:
Нормализуйте колонки, которые вы отметили в квизе.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.

Сдайте код, который модифицирует `df_train` и `df_test` так, чтобы численные колонки из прошлого пункта стали нормированы.

In [28]:
scaler = StandardScaler()

In [29]:
scaler.fit(df_train[['age', 'bmi', 'charges']])

StandardScaler()

In [30]:
df_train[['age', 'bmi', 'charges']] = scaler.transform(df_train[['age', 'bmi', 'charges']])
df_test[['age', 'bmi', 'charges']] = scaler.transform(df_test[['age', 'bmi', 'charges']])

In [31]:
df_train.to_csv('train.csv')

In [32]:
df_test.to_csv('test.csv')

In [22]:
y_train = df_train.pop("charges")
y_test = df_test.pop("charges")

### Задание №5
Реализуйте функцию, считающую `MSE` метрику.

Ваша функция должна уметь принимать torch.Tensor, numpy-массивы и pd.Series.

In [10]:
import torch

In [11]:
# Определяем устройство: если доступен GPU, используем cuda, иначе CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Используем устройство", device)


Используем устройство cpu


In [12]:
def metric(preds, y):

    for i in (preds, y):
        if type(i) == np.ndarray:
            i = torch.from_numpy(preds)
        elif type(i) == pd.Series:
            i = torch.tensor(i.values)
    MSE = sum(((preds - y)**2) / y.shape[0])
    return MSE

### Задание №6
Реализуйте бейзлайн на `LinearRegression` и `GradientBoostingRegressor`, отправьте метрики в ЛМС.

Используйте гиперпараметры по-умолчанию в обоих моделях.

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression


In [16]:
df_train.drop(columns=['sex', 'region', 'smoker'], axis=1, inplace=True)
df_test.drop(columns=['sex', 'region', 'smoker'], axis=1, inplace=True)

In [17]:
boost = GradientBoostingRegressor()
linear = LinearRegression()

In [18]:
boost.fit(df_train, y_train)
linear.fit(df_train, y_train)

LinearRegression()

In [19]:
print(metric(boost.predict(df_test), y_test))
print(metric(linear.predict(df_test), y_test))

0.11499598580020742
0.2230241963394205


### Задание №7
Вычислите среднее значение целевой переменной на тренировочной выборке (train).

Подсчитайте MSE при константном предсказании этим средним и отправьте его в ЛМС.

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
target_mean = np.mean(y_train)

In [22]:
dummy_model = np.full_like(y_test, target_mean)

In [23]:
metric(target_mean, y_test)

1.115550592874472

### Задание №8
Создайте сеть, состоящую из одного слоя Linear, залейте в лмс код, который в `model` запишет вашу модель

В качестве признаков используйте все колонки в текущем датасете, за исключением таргета

In [24]:
import torch.nn as nn


def build_model():

    mod = nn.Linear(in_features=11, out_features=1)
    return mod

model = build_model()

In [25]:
import torch

torch.random.manual_seed(seed)
t_x_train = torch.from_numpy(df_train.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_train = torch.from_numpy(y_train.to_numpy()).to(dtype=torch.float32).view(-1, 1)
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32).view(-1, 1)

In [26]:
t_x_train.shape

torch.Size([1071, 11])

In [27]:
t_y_train.shape

torch.Size([1071, 1])

### Задание №9
Напишите функцию `train_loop`, которая будет учить модель по данным на 2к итераций.
Считайте, что данные уже хранятся в переменных `t_x_train`, `t_y_train`.
Ваша функция `train_loop` должна вернуть список из лоссов на каждой итерации (т.е. список длины 2000).

Используйте `learning_rate=1e-2` в оптимизаторе.

Для простоты за одну итерацию делайте проход вперед и проход назад на всех наших обучающих данных.
Это будет полный градиентный спуск (не по батчам) - можем себе позволить, данных немного.

_Подсказка 1_: Вам не обязательно учить модель на видеокарте, CPU будет достаточно.

_Подсказка 2_: `tqdm` - это библиотека, которая рисует прогресс итераций

In [28]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Глобальные переменные t_x_train, t_y_train должны быть доступны в среде
def train_loop(model: nn.Module) -> list[float]:
    losses = []
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=1e-2)

    for _ in tqdm(range(2000)):  # Прогресс-бар для наглядности
        optimizer.zero_grad()
        predictions = model(t_x_train)
        loss = criterion(predictions, t_y_train)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())  # Сохраняем loss для каждой итерации
    
    return losses


### Задание №10
Обучите модель, состоящую из одного слоя `Linear`.
Приложите в ЛМС метрику `MSE` на тестовых данных.
Используйте `learning_rate=1e-2` в оптимизаторе.

Когда будете тестировать, не забудьте перенести тестовые данные в `torch.Tensor`

In [36]:
train_loop(model)

100%|██████████| 2000/2000 [00:00<00:00, 2148.29it/s]


[1.27213454246521,
 1.2554354667663574,
 1.24014413356781,
 1.2260209321975708,
 1.2130261659622192,
 1.201027750968933,
 1.1899504661560059,
 1.1796804666519165,
 1.1701879501342773,
 1.1613634824752808,
 1.1531615257263184,
 1.1454819440841675,
 1.138307809829712,
 1.1315735578536987,
 1.1252400875091553,
 1.119249939918518,
 1.11359703540802,
 1.1082377433776855,
 1.1031432151794434,
 1.098306655883789,
 1.0937085151672363,
 1.0893282890319824,
 1.0851669311523438,
 1.0811935663223267,
 1.0773717164993286,
 1.0736899375915527,
 1.070155143737793,
 1.0667905807495117,
 1.0635557174682617,
 1.0604358911514282,
 1.0574589967727661,
 1.054590106010437,
 1.0518134832382202,
 1.0491206645965576,
 1.0464991331100464,
 1.0439547300338745,
 1.041480541229248,
 1.0390862226486206,
 1.036756992340088,
 1.0345004796981812,
 1.0322985649108887,
 1.0301318168640137,
 1.0280269384384155,
 1.0259881019592285,
 1.0239949226379395,
 1.0220483541488647,
 1.020148754119873,
 1.018275499343872,
 1.01644

In [30]:
import torch.nn.functional as F

In [37]:
# Считаем финальную метрику на тестовых данных
with torch.no_grad():
    # Помним, что все тензоры должны быть на одном устройстве
    out = model(t_x_test)
    mse_loss = F.mse_loss(out, t_y_test).item()
    print("Финальный MSE-лосс:", mse_loss)

Финальный MSE-лосс: 0.14732885360717773


In [38]:
# from torchmetrics.functional import mean_squared_error

with torch.no_grad():
    predictions = model(t_x_test)
    mse = metric(predictions, t_y_test)

# print(mse.item())  # Отправь этот результат в чекер

In [39]:
sum(mse) / len(mse)

tensor(0.1473)

### Задание №11
Вам необходимо усложнить существующую нейронную сеть, добавив один скрытый слой.

Используйте следующие параметры:

Размерность скрытого слоя: 6 нейронов, функция активации -  `ReLU`

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [35]:
def build_model():

    input_dim = t_x_train.shape[1]  # Количество признаков\
    transform = nn.Sequential(
        nn.Linear(in_features=input_dim, out_features=6),
        nn.ReLU(),
        nn.Linear(in_features=6, out_features=1)
    )



    return transform

model = build_model()

### Задание №12
Приложите в ЛМС метрику качества этой сети после 2к итераций обучения.
Эту модель можно обучить на CPU, не обязательно на видеокарте.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

In [41]:
train_loop(model)

100%|██████████| 2000/2000 [00:01<00:00, 1362.80it/s]


[1.066694736480713,
 1.0629256963729858,
 1.0593948364257812,
 1.0560872554779053,
 1.0529886484146118,
 1.050086259841919,
 1.047365427017212,
 1.0448161363601685,
 1.042427897453308,
 1.0401896238327026,
 1.0380923748016357,
 1.0361274480819702,
 1.034286618232727,
 1.0325602293014526,
 1.0309422016143799,
 1.0294252634048462,
 1.0280017852783203,
 1.0266669988632202,
 1.0254151821136475,
 1.0242412090301514,
 1.023139476776123,
 1.0221054553985596,
 1.021134614944458,
 1.0202226638793945,
 1.0193657875061035,
 1.0185601711273193,
 1.0178027153015137,
 1.0170902013778687,
 1.0164191722869873,
 1.0157877206802368,
 1.0151926279067993,
 1.0146321058273315,
 1.0141043663024902,
 1.0136067867279053,
 1.0131374597549438,
 1.012694001197815,
 1.012275218963623,
 1.011879324913025,
 1.011505126953125,
 1.0111510753631592,
 1.0108157396316528,
 1.0104973316192627,
 1.0101953744888306,
 1.0099085569381714,
 1.0096360445022583,
 1.0093770027160645,
 1.0091303586959839,
 1.0088951587677002,
 1.

In [42]:
# Считаем финальную метрику на тестовых данных
with torch.no_grad():
    # Помним, что все тензоры должны быть на одном устройстве
    out = model(t_x_test)
    mse_loss = F.mse_loss(out, t_y_test).item()
    print("Финальный MSE-лосс:", mse_loss)

Финальный MSE-лосс: 0.2039656788110733


### Задание №13
Добавьте дополнительные слои в нейронную сеть

Вам необходимо усложнить нейронную сеть, добавив еще 2-3 скрытых слоя с такими же размерностями, как в предыдущем задании.

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [43]:
def build_model():

    input_dim = t_x_train.shape[1]  # Количество признаков\
    transform = nn.Sequential(
        nn.Linear(in_features=input_dim, out_features=6),
        nn.ReLU(),
        nn.Linear(in_features=6, out_features=6),
        nn.ReLU(),
        nn.Linear(in_features=6, out_features=1),
    )



    return transform

model = build_model()

### Задание №14
Приложите в ЛМС метрику качества после 2к итераций обучения.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

Эту модель можно обучить на CPU, не обязательно на видеокарте.

In [44]:
train_loop(model)

100%|██████████| 2000/2000 [00:01<00:00, 1753.05it/s]


[1.1064765453338623,
 1.0976662635803223,
 1.0894132852554321,
 1.081684947013855,
 1.0744260549545288,
 1.0676114559173584,
 1.0612142086029053,
 1.055219054222107,
 1.0495846271514893,
 1.0442914962768555,
 1.0393043756484985,
 1.034608006477356,
 1.0301827192306519,
 1.0260095596313477,
 1.02207350730896,
 1.0183587074279785,
 1.0148520469665527,
 1.0115478038787842,
 1.008412480354309,
 1.0054423809051514,
 1.0026235580444336,
 0.9999462962150574,
 0.9974046349525452,
 0.9949841499328613,
 0.992673933506012,
 0.9904654026031494,
 0.9883548617362976,
 0.9863366484642029,
 0.9844092130661011,
 0.9825658202171326,
 0.9807982444763184,
 0.9790967106819153,
 0.9774633646011353,
 0.9758862853050232,
 0.9743503928184509,
 0.9728628993034363,
 0.9714182019233704,
 0.9700085520744324,
 0.9686456918716431,
 0.9673232436180115,
 0.9660345911979675,
 0.9647795557975769,
 0.9635521769523621,
 0.9623516201972961,
 0.9611761569976807,
 0.9600266814231873,
 0.9589015245437622,
 0.957796037197113,


In [45]:
# Считаем финальную метрику на тестовых данных
with torch.no_grad():
    # Помним, что все тензоры должны быть на одном устройстве
    out = model(t_x_test)
    mse_loss = F.mse_loss(out, t_y_test).item()
    print("Финальный MSE-лосс:", mse_loss)

Финальный MSE-лосс: 0.1352768838405609
